In [1]:
#!pip install simplet5
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"

In [7]:
%cd /content/drive/MyDrive/Kaggle/

/content/drive/MyDrive/Kaggle


In [6]:
!kaggle datasets download -d nmaria/news-t5-model

 99% 678M/685M [00:09<00:00, 119MB/s] 
100% 685M/685M [00:09<00:00, 75.6MB/s]


In [7]:
!unzip /content/drive/MyDrive/Kaggle/news-t5-model.zip

Archive:  /content/drive/MyDrive/Kaggle/news-t5-model.zip
  inflating: outputs/checkpoint-185548-epoch-2/config.json  
  inflating: outputs/checkpoint-185548-epoch-2/model_args.json  
  inflating: outputs/checkpoint-185548-epoch-2/optimizer.pt  
  inflating: outputs/checkpoint-185548-epoch-2/pytorch_model.bin  
  inflating: outputs/checkpoint-185548-epoch-2/scheduler.pt  
  inflating: outputs/checkpoint-185548-epoch-2/special_tokens_map.json  
  inflating: outputs/checkpoint-185548-epoch-2/spiece.model  
  inflating: outputs/checkpoint-185548-epoch-2/tokenizer_config.json  
  inflating: outputs/checkpoint-185548-epoch-2/training_args.bin  
  inflating: outputs/checkpoint-92774-epoch-1/config.json  
  inflating: outputs/checkpoint-92774-epoch-1/model_args.json  
  inflating: outputs/checkpoint-92774-epoch-1/optimizer.pt  
  inflating: outputs/checkpoint-92774-epoch-1/pytorch_model.bin  
  inflating: outputs/checkpoint-92774-epoch-1/scheduler.pt  
  inflating: outputs/checkpoint-92774-ep

In [11]:
!kaggle datasets download -d nmaria/lentaru-20202021

 93% 136M/147M [00:01<00:00, 82.4MB/s]
100% 147M/147M [00:01<00:00, 91.3MB/s]


In [12]:
!unzip /content/drive/MyDrive/Kaggle/lentaru-20202021.zip

Archive:  /content/drive/MyDrive/Kaggle/lentaru-20202021.zip
  inflating: lenta-ru-news.csv       
  inflating: lenta-ru-news_1.csv     


In [20]:
import pandas as pd

df1 = pd.read_csv('/content/drive/MyDrive/Kaggle/lenta-ru-news.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Kaggle/lenta-ru-news_1.csv')
df =  pd.concat([df1, df2], ignore_index=True)
df['text'] = df['text'].apply(lambda x: 'grammar: ' + x)
texts = df['text'].tolist()[-20617:]

In [ ]:
import shutil
import os

os.listdir('./cache_dir/cointegrated_rut5-small_cached_512185547')

In [ ]:
for f in os.listdir('./cache_dir'):
    shutil.copy(os.path.join('./cache_dir', f), os.path.join('./', f))

In [17]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
# !pip install transformers sentencepiece
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/Kaggle/outputs")
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Kaggle/outputs")

inputs = tokenizer(texts, return_tensors='pt')
with torch.no_grad():
    hypotheses = model.generate(
        **inputs, 
        do_sample=True, top_p=0.95, num_return_sequences=1, 
        repetition_penalty=2.5,
        max_length=512,
    )
#for h in hypotheses:
#    print(tokenizer.decode(h, skip_special_tokens=True))

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


KeyboardInterrupt: ignored

In [ ]:
from pprint import pprint
import pandas as pd
from simpletransformers.t5 import T5Model

'''model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 512,
    "train_batch_size": 2,
    "num_train_epochs": 2,
    "max_length": 512,
    "length_penalty": 10,
    #"save_eval_checkpoints": True,
    "save_steps": -1,
    "use_multiprocessing": False,
    "evaluate_during_training": False,
    "fp16": False,
    #"wandb_project": "GEC with T5",
}'''
model = T5Model('mt5', "/content/drive/MyDrive/Kaggle/outputs", use_cuda=True, use_multiprocessing=False)

#eval_df = pd.read_csv('../input/noisy-news/test_t5.csv', header=0, names=['ind', 'input_text', 'target_text'], usecols=['input_text', 'target_text']) 
#eval_df['input_text'] = eval_df['input_text'].apply(lambda x: x[9:])
#eval_df['prefix'] = ['grammar'] * len(eval_df)
preds = model.predict(texts)

pred_df = pd.DataFrame({'preds': preds, 'input': texts})
pred_df.to_csv('/content/drive/MyDrive/Kaggle/pred_t5_full.csv', index=False)

Generating outputs:   0%|          | 0/2578 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3538: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [ ]:
preds[0]

In [ ]:
eval_df['target_text'][0]

In [ ]:
pred_df.gold[0]